## Demo: process synchronization
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

Next to the synchronization of activities using controll structures like **while** and **sequence** used within a single process, OpenCLSim allows to synchronize different processes using **start_events**. Start_events can be specified using the expression language as documented to **delay** the execution of a process. Control structures do not delay the execution but explicitly start the execution when possible. The difference is very well visible when you compare the activity log in this demo with the one from the sequence demo.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes
In this example we won't use object classes

#### 3. Create objects
##### 3.1. Create site object(s)
No site objects are created

##### 3.2. Create vessel object(s)
No vessel objects are created

##### 3.3 Create activity/activities
The two activities are started in parallel, but the execution of Activity2 is delayed until Activity1 has been completed. To easier see how the two activities are interrelated a reporting activity is added.

In [3]:
# initialise registry
registry = {}

In [4]:
# create a reporting activity
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    registry=registry,
    duration=0,
)

# create two basic activities
activity1 = model.BasicActivity(
    env=my_env,
    name="Activity1",
    registry=registry,
    additional_logs=[reporting_activity],
    duration=20,
)
activity2 = model.BasicActivity(
    env=my_env,
    name="Activity2",
    registry=registry,
    additional_logs=[reporting_activity],
    start_event=[{"type": "activity", "name": "Activity1", "state": "done"}],
    duration=40,
)

#### 4. Register processes and run simpy

In [5]:
# initate the simpy processes defined in activity1 and activity2 and run simpy
model.register_processes([reporting_activity, activity1, activity2])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs

In [6]:
display(plot.get_log_dataframe(reporting_activity, [reporting_activity, activity1, activity2]))

,Activity,Timestamp,ActivityState,type,ref
0,Activity1,1970-01-01 00:00:00,START,additional log,89a816a0-b39b-416e-884a-86d820716c10
1,Reporting activity,1970-01-01 00:00:00,START,NaN,NaN
2,Reporting activity,1970-01-01 00:00:00,STOP,NaN,NaN
3,Activity1,1970-01-01 00:00:20,STOP,additional log,89a816a0-b39b-416e-884a-86d820716c10
4,Activity2,1970-01-01 00:00:00,WAIT_START,additional log,251b4b7d-e411-4aa7-a676-66152b6984fe
5,Activity2,1970-01-01 00:00:20,WAIT_STOP,additional log,251b4b7d-e411-4aa7-a676-66152b6984fe
6,Activity2,1970-01-01 00:00:20,START,additional log,251b4b7d-e411-4aa7-a676-66152b6984fe
7,Activity2,1970-01-01 00:01:00,STOP,additional log,251b4b7d-e411-4aa7-a676-66152b6984fe


Both activities start at the same time. Activity2 gets into a waiting state, which ends, when Activity1 ends. Then Activity2 is executed.

##### 5.2 Visualise gantt charts

In [7]:
plot.get_gantt_chart([reporting_activity, activity1, activity2])